In [ ]:
import torch
import pandas as pd
import pyarrow.parquet as pq
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pathlib import Path

MODEL = "ProsusAI/finbert"
BATCH_SIZE = 64

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.eval()

if torch.cuda.is_available():
    model = model.cuda()

labels = ["negative", "neutral", "positive"]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Mounted at /content/drive


In [4]:
dataset = pq.ParquetDataset("/content/drive/MyDrive/Project_Data/data_for_finbert/")

In [5]:
from tqdm import tqdm

results = []

for fragment in dataset.fragments:
    table = fragment.to_table()
    df = table.to_pandas()

    texts = df["Lsa_summary"].tolist()

    # Progress bar for batches only
    for i in tqdm(range(0, len(texts), BATCH_SIZE), desc="Processing batches"):
        batch = texts[i:i + BATCH_SIZE]

        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            return_tensors="pt"
        )

        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.softmax(outputs.logits, dim=1)

        for j in range(len(batch)):
            result = {
                "No": df.iloc[i + j]["No"]
            }

            # Add all label probabilities
            for label_idx, label_name in enumerate(labels):
                result[label_name] = probs[j][label_idx].item()

            # Optionally add the predicted sentiment
            result["predicted_sentiment"] = labels[torch.argmax(probs[j]).item()]

            results.append(result)

Processing batches: 100%|██████████| 3/3 [00:03<00:00,  1.07s/it]


In [6]:
pd.DataFrame(results).to_parquet(
    "/content/drive/MyDrive/Project_Data/sentiment_data.parquet",
    index=False
)